# Day11_2: 감성 분석 (Sentiment Analysis) - 정답

## 📚 학습 목표

**Part 1: 사전 기반 감성 분석**
1. 감성 분석의 개념과 비즈니스 활용 이해하기
2. VADER 감성 분석기로 영어 텍스트 분석하기
3. KNU 한국어 감성 사전 활용법 배우기
4. 커스텀 감성 사전 구축하기
5. 긍정/부정/중립 분류 구현하기

**Part 2: 감성 분석 시각화와 응용**
1. pandas로 감성 점수 집계 및 분석하기
2. Plotly로 감성 분포 시각화하기 (파이/막대 차트)
3. 시계열 감성 트렌드 분석하기
4. 머신러닝 기반 감성 분석 소개

---

In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import re
from collections import Counter
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# VADER 감성 분석기 (영어용)
try:
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
    vader_available = True
    print("VADER 감성 분석기 로드 완료")
except ImportError:
    vader_available = False
    print("VADER 미설치. pip install vaderSentiment 로 설치하세요.")

print("라이브러리 임포트 완료")

In [ ]:
# 한국어 감성 사전 정의 (퀴즈에서 공통 사용)
korean_sentiment_dict = {
    # 긍정 단어
    '좋다': 2, '좋아': 2, '좋은': 2, '좋아요': 2,
    '훌륭하다': 2, '훌륭한': 2, '훌륭해요': 2,
    '최고': 2, '최고다': 2, '최고예요': 2,
    '만족': 2, '만족스럽다': 2, '만족해요': 2, '만족합니다': 2,
    '추천': 1, '추천해요': 1, '추천합니다': 1,
    '맛있다': 2, '맛있어요': 2, '맛있는': 2,
    '친절하다': 2, '친절해요': 2, '친절한': 2,
    '빠르다': 1, '빨라요': 1, '빠른': 1,
    '편하다': 1, '편해요': 1, '편한': 1,
    '깨끗하다': 1, '깨끗해요': 1, '깨끗한': 1,
    # 부정 단어
    '나쁘다': -2, '나빠요': -2, '나쁜': -2,
    '싫다': -2, '싫어요': -2, '싫은': -2,
    '별로': -1, '별로다': -1, '별로예요': -1,
    '실망': -2, '실망스럽다': -2, '실망이에요': -2,
    '불만족': -2, '불만': -2,
    '최악': -2, '최악이다': -2, '최악이에요': -2,
    '느리다': -1, '느려요': -1, '느린': -1,
    '불친절': -2, '불친절하다': -2,
    '불편하다': -1, '불편해요': -1,
    # 강조어
    '매우': 1.5, '정말': 1.5, '너무': 1.3, '아주': 1.3
}

# 색상 정의
colors = {'긍정': '#2ecc71', '부정': '#e74c3c', '중립': '#95a5a6'}

print("감성 사전 및 색상 정의 완료")

---

## 🎯 실습 퀴즈 정답

---

### Q1. 감성 점수로 분류하기 ⭐

**문제**: 다음 감성 점수들을 긍정/부정/중립으로 분류하는 함수를 작성하세요.

- 0.05 이상: 긍정
- -0.05 이하: 부정
- 그 외: 중립

```python
scores = [0.8, -0.3, 0.02, -0.7, 0.5, 0.0, -0.04]
```

**기대 결과**: `['긍정', '부정', '중립', '부정', '긍정', '중립', '중립']`

In [ ]:
# Q1 정답

scores = [0.8, -0.3, 0.02, -0.7, 0.5, 0.0, -0.04]

# 1. 분류 함수 정의
def classify_sentiment(score):
    """감성 점수를 긍정/부정/중립으로 분류"""
    if score >= 0.05:
        return '긍정'
    elif score <= -0.05:
        return '부정'
    else:
        return '중립'

# 2. 리스트 컴프리헨션으로 분류
sentiments = [classify_sentiment(s) for s in scores]

# 3. 결과 출력
print("감성 분류 결과:")
print(sentiments)

# 점수와 함께 출력
for score, sentiment in zip(scores, sentiments):
    print(f"  {score:6.2f} → {sentiment}")

In [ ]:
# 테스트
expected = ['긍정', '부정', '중립', '부정', '긍정', '중립', '중립']
assert sentiments == expected, f"결과 불일치: {sentiments}"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
조건문으로 점수 범위에 따라 감성을 분류합니다. 경계값(0.05, -0.05) 처리에 주의합니다.

**핵심 개념**:
- if-elif-else 조건문
- 리스트 컴프리헨션
- 함수 정의와 활용

**대안 솔루션**:
```python
# numpy로 벡터화
import numpy as np
arr = np.array(scores)
result = np.where(arr >= 0.05, '긍정', np.where(arr <= -0.05, '부정', '중립'))
```

**흔한 실수**:
- ❌ 경계값 처리 오류: `> 0.05` vs `>= 0.05`
- ❌ 중립 범위 누락: -0.05 < x < 0.05 사이 처리

**실무 팁**:
임계값(0.05)은 데이터에 따라 조정할 수 있습니다. 보통 0.05 ~ 0.1 사이를 사용합니다.

---

### Q2. 감성 사전으로 점수 계산하기 ⭐⭐

**문제**: 주어진 감성 사전을 사용하여 텍스트의 감성 점수를 계산하세요.

```python
simple_dict = {'좋다': 1, '나쁘다': -1, '최고': 2, '최악': -2, '보통': 0}
texts = ['이 음식 최고다', '서비스가 나쁘다', '그냥 보통이다']
```

In [ ]:
# Q2 정답

simple_dict = {'좋다': 1, '나쁘다': -1, '최고': 2, '최악': -2, '보통': 0}
texts = ['이 음식 최고다', '서비스가 나쁘다', '그냥 보통이다']

# 1. 감성 점수 계산 함수
def calculate_sentiment_score(text, sentiment_dict):
    """텍스트에서 감성 점수 계산"""
    score = 0
    found_words = []
    
    for word, word_score in sentiment_dict.items():
        if word in text:
            score += word_score
            found_words.append((word, word_score))
    
    return score, found_words

# 2. 각 텍스트 분석
print("감성 점수 계산 결과")
print("=" * 50)

results = []
for text in texts:
    score, words = calculate_sentiment_score(text, simple_dict)
    results.append(score)
    print(f"\n텍스트: {text}")
    print(f"  발견된 단어: {words}")
    print(f"  감성 점수: {score}")

print(f"\n점수 리스트: {results}")

In [ ]:
# 테스트
assert results[0] == 2, "첫 번째 텍스트 점수 오류 (최고 = 2)"
assert results[1] == -1, "두 번째 텍스트 점수 오류 (나쁘다 = -1)"
assert results[2] == 0, "세 번째 텍스트 점수 오류 (보통 = 0)"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
감성 사전의 각 단어가 텍스트에 포함되어 있는지 확인하고, 해당 점수를 합산합니다.

**핵심 개념**:
- 딕셔너리 순회 (items())
- 문자열 포함 여부 확인 (in 연산자)
- 점수 누적

**대안 솔루션**:
```python
# 정규표현식으로 단어 경계 처리
import re
for word, score in sentiment_dict.items():
    if re.search(rf'{word}', text):
        total += score
```

**흔한 실수**:
- ❌ 부분 일치 문제: '좋다'가 '좋다운'에도 매칭
- ❌ 대소문자/공백 미처리

**실무 팁**:
실제 구현에서는 형태소 분석기(KoNLPy)로 정확한 단어 추출 후 매칭하는 것이 좋습니다.

---

### Q3. pandas로 감성 분포 집계하기 ⭐⭐

**문제**: 다음 리뷰 데이터에서 감성별 리뷰 수와 비율을 계산하세요.

```python
reviews_df = pd.DataFrame({
    'review': ['좋아요', '별로예요', '최고!', '그냥 그래요', '실망이에요', '만족해요'],
    'sentiment': ['긍정', '부정', '긍정', '중립', '부정', '긍정']
})
```

In [ ]:
# Q3 정답

reviews_df = pd.DataFrame({
    'review': ['좋아요', '별로예요', '최고!', '그냥 그래요', '실망이에요', '만족해요'],
    'sentiment': ['긍정', '부정', '긍정', '중립', '부정', '긍정']
})

# 1. 감성별 리뷰 수 집계
sentiment_counts = reviews_df['sentiment'].value_counts()

# 2. 비율 계산
sentiment_pct = reviews_df['sentiment'].value_counts(normalize=True) * 100

# 3. 결과를 DataFrame으로 정리
result_df = pd.DataFrame({
    '리뷰 수': sentiment_counts,
    '비율(%)': sentiment_pct.round(1)
})

print("감성별 분포")
print("=" * 30)
print(result_df)

# 4. 총계
print(f"\n총 리뷰 수: {len(reviews_df)}개")

In [ ]:
# 테스트
assert sentiment_counts['긍정'] == 3, "긍정 개수 오류"
assert sentiment_counts['부정'] == 2, "부정 개수 오류"
assert sentiment_counts['중립'] == 1, "중립 개수 오류"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
`value_counts()`로 카테고리별 개수를 집계하고, `normalize=True`로 비율을 계산합니다.

**핵심 개념**:
- `value_counts()`: 고유값별 개수
- `normalize=True`: 비율로 반환
- DataFrame 생성으로 깔끔한 출력

**대안 솔루션**:
```python
# groupby 사용
reviews_df.groupby('sentiment').size()

# 크로스탭
pd.crosstab(reviews_df['sentiment'], columns='count')
```

**흔한 실수**:
- ❌ 비율 계산 시 100을 곱하지 않음
- ❌ 반올림 없이 긴 소수점 출력

**실무 팁**:
비율은 항상 소수점 1자리로 반올림하고, 합이 100%인지 확인합니다.

---

### Q4. Plotly 파이 차트 그리기 ⭐⭐⭐

**문제**: Q3의 감성 분포를 Plotly 파이 차트로 시각화하세요.

**요구사항**:
- 긍정=초록, 부정=빨강, 중립=회색
- 퍼센트와 레이블 표시
- 도넛 차트 형태 (hole=0.3)

In [ ]:
# Q4 정답

# Q3의 데이터 사용
sentiment_counts = reviews_df['sentiment'].value_counts()

# 1. 색상 정의
colors = {'긍정': '#2ecc71', '부정': '#e74c3c', '중립': '#95a5a6'}

# 2. 파이 차트 생성
fig = px.pie(
    values=sentiment_counts.values,
    names=sentiment_counts.index,
    title='리뷰 감성 분포',
    color=sentiment_counts.index,
    color_discrete_map=colors,
    hole=0.3  # 도넛 차트
)

# 3. 텍스트 포맷 설정
fig.update_traces(
    textposition='inside',
    textinfo='percent+label',
    textfont_size=14
)

# 4. 레이아웃 설정
fig.update_layout(
    width=600,
    height=450,
    showlegend=True
)

fig.show()

In [ ]:
# 테스트
assert fig is not None, "차트 생성 실패"
print("테스트 통과! 파이 차트가 성공적으로 생성되었습니다.")

### 💡 풀이 설명

**접근 방법**:
`px.pie()`로 기본 파이 차트를 생성하고, `color_discrete_map`으로 감성별 색상을 지정합니다.

**핵심 개념**:
- `px.pie()`: 파이 차트 생성
- `hole=0.3`: 도넛 차트 (중앙 구멍)
- `color_discrete_map`: 카테고리별 색상 매핑
- `textinfo`: 표시할 정보 (percent, label, value)

**대안 솔루션**:
```python
# go.Pie 직접 사용
fig = go.Figure(data=[go.Pie(
    labels=sentiment_counts.index,
    values=sentiment_counts.values,
    hole=0.3,
    marker_colors=[colors[s] for s in sentiment_counts.index]
)])
```

**흔한 실수**:
- ❌ color_discrete_map 키와 데이터 불일치
- ❌ textinfo에 잘못된 옵션 사용

**실무 팁**:
감성 분석에서 긍정=초록, 부정=빨강은 직관적인 색상 약속입니다. 일관성 있게 사용하세요.

---

### Q5. 카테고리별 감성 비교 막대 차트 ⭐⭐⭐

**문제**: 다음 데이터를 사용하여 카테고리별 평균 감성 점수 막대 차트를 그리세요.

```python
category_data = pd.DataFrame({
    'category': ['음식', '음식', '서비스', '서비스', '가격', '가격'],
    'sentiment_score': [0.8, 0.6, -0.3, 0.2, 0.4, -0.5]
})
```

In [ ]:
# Q5 정답

category_data = pd.DataFrame({
    'category': ['음식', '음식', '서비스', '서비스', '가격', '가격'],
    'sentiment_score': [0.8, 0.6, -0.3, 0.2, 0.4, -0.5]
})

# 1. 카테고리별 평균 점수 계산
category_avg = category_data.groupby('category')['sentiment_score'].mean().sort_values()

print("카테고리별 평균 감성 점수")
print(category_avg)

# 2. 색상 설정 (양수=초록, 음수=빨강)
bar_colors = ['#2ecc71' if x >= 0 else '#e74c3c' for x in category_avg.values]

# 3. 막대 차트 생성
fig = go.Figure()

fig.add_trace(go.Bar(
    x=category_avg.index,
    y=category_avg.values,
    marker_color=bar_colors,
    text=[f'{v:.2f}' for v in category_avg.values],
    textposition='outside'
))

# 4. 0 기준선 추가
fig.add_hline(y=0, line_dash='dash', line_color='gray')

# 5. 레이아웃 설정
fig.update_layout(
    title='카테고리별 평균 감성 점수',
    xaxis_title='카테고리',
    yaxis_title='감성 점수',
    yaxis=dict(range=[-1, 1]),
    width=600,
    height=400
)

fig.show()

In [ ]:
# 테스트
assert category_avg['음식'] == 0.7, "음식 평균 점수 오류"
assert category_avg['서비스'] == -0.05, "서비스 평균 점수 오류"
assert category_avg['가격'] == -0.05, "가격 평균 점수 오류"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. groupby로 카테고리별 평균 계산
2. 점수 부호에 따라 색상 리스트 생성
3. go.Bar로 막대 차트 생성
4. add_hline으로 기준선 추가

**핵심 개념**:
- `groupby().mean()`: 그룹별 평균
- 리스트 컴프리헨션으로 조건부 색상
- `add_hline()`: 수평 기준선

**대안 솔루션**:
```python
# px.bar 사용
fig = px.bar(
    x=category_avg.index,
    y=category_avg.values,
    color=category_avg.values > 0,
    color_discrete_map={True: '#2ecc71', False: '#e74c3c'}
)
```

**흔한 실수**:
- ❌ 정렬 없이 막대 차트 생성
- ❌ y축 범위 미설정으로 비교 어려움

**실무 팁**:
비교 차트에서는 정렬(오름차순/내림차순)과 기준선이 인사이트 도출에 매우 중요합니다.

---

## Part 2 심화 퀴즈 정답

---

### Q6. 강조어 처리하는 감성 분석 함수 ⭐⭐⭐

**문제**: 강조어(매우, 정말, 너무)가 있으면 점수를 1.5배 하는 감성 분석 함수를 작성하세요.

```python
sentiment_dict = {'좋다': 1, '나쁘다': -1}
texts = ['좋다', '매우 좋다', '정말 나쁘다', '나쁘다']
```

**기대 결과**: `[1, 1.5, -1.5, -1]`

In [ ]:
# Q6 정답

sentiment_dict = {'좋다': 1, '나쁘다': -1}
texts = ['좋다', '매우 좋다', '정말 나쁘다', '나쁘다']

def analyze_with_intensifiers(text, sentiment_dict):
    """
    강조어를 처리하는 감성 분석
    강조어가 있으면 점수를 1.5배
    """
    # 1. 강조어 리스트
    intensifiers = ['매우', '정말', '너무']
    
    # 2. 강조어 확인
    multiplier = 1.0
    for intensifier in intensifiers:
        if intensifier in text:
            multiplier = 1.5
            break
    
    # 3. 감성 단어 점수 계산
    score = 0
    for word, word_score in sentiment_dict.items():
        if word in text:
            score += word_score * multiplier
    
    return score

# 4. 결과 계산
results = [analyze_with_intensifiers(text, sentiment_dict) for text in texts]

print("강조어 처리 감성 분석 결과")
for text, score in zip(texts, results):
    print(f"  {text}: {score}")

print(f"\n결과 리스트: {results}")

In [ ]:
# 테스트
expected = [1, 1.5, -1.5, -1]
assert results == expected, f"결과 불일치: {results} vs {expected}"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. 먼저 강조어 존재 여부를 확인하여 배수(multiplier) 결정
2. 감성 단어 점수에 배수를 곱함

**핵심 개념**:
- 강조어(intensifier): 감정 강도를 높이는 부사
- 배수 적용: 기본 점수 * 배수

**대안 솔루션**:
```python
# any()와 삼항 연산자 사용
multiplier = 1.5 if any(i in text for i in intensifiers) else 1.0
```

**흔한 실수**:
- ❌ 강조어와 감성 단어 순서 잘못 처리
- ❌ 배수를 더하기로 처리 (곱하기가 맞음)

**실무 팁**:
VADER는 강조어뿐 아니라 대문자, 느낌표, 이모지도 처리합니다. 커스텀 구현 시 참고하세요.

---

### Q7. 제품별 긍정률 계산 및 순위 ⭐⭐⭐

**문제**: 제품별 긍정률(긍정 리뷰 / 전체 리뷰 * 100)을 계산하고 높은 순서로 정렬하세요.

In [ ]:
# Q7 정답

product_reviews = pd.DataFrame({
    'product': ['A', 'A', 'A', 'B', 'B', 'B', 'C', 'C', 'C', 'C'],
    'sentiment': ['긍정', '긍정', '부정', '긍정', '부정', '부정', '긍정', '긍정', '긍정', '중립']
})

# 1. 제품별 전체 리뷰 수
total_by_product = product_reviews.groupby('product').size()

# 2. 제품별 긍정 리뷰 수
positive_by_product = product_reviews[product_reviews['sentiment'] == '긍정'].groupby('product').size()

# 3. 긍정률 계산
positive_rate = (positive_by_product / total_by_product * 100).fillna(0)

# 4. 결과 DataFrame 생성 및 정렬
result = pd.DataFrame({
    '전체 리뷰': total_by_product,
    '긍정 리뷰': positive_by_product,
    '긍정률(%)': positive_rate.round(1)
}).sort_values('긍정률(%)', ascending=False)

print("제품별 긍정률 순위")
print("=" * 40)
print(result)

# 5. 순위 출력
print("\n순위:")
for rank, (product, row) in enumerate(result.iterrows(), 1):
    print(f"  {rank}위: {product} ({row['긍정률(%)']:.1f}%)")

In [ ]:
# 테스트
assert positive_rate['C'] == 75.0, "C 긍정률 오류"
assert positive_rate['A'] == pytest.approx(66.67, rel=0.1), "A 긍정률 오류"
assert positive_rate['B'] == pytest.approx(33.33, rel=0.1), "B 긍정률 오류"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. 전체 리뷰 수와 긍정 리뷰 수를 각각 groupby로 집계
2. 긍정률 = 긍정 / 전체 * 100
3. 내림차순 정렬

**핵심 개념**:
- 조건부 필터링 후 groupby
- Series 나누기 (자동 인덱스 정렬)
- sort_values(ascending=False)

**대안 솔루션**:
```python
# 한 번의 groupby로 처리
product_reviews.groupby('product').agg(
    긍정률=('sentiment', lambda x: (x == '긍정').sum() / len(x) * 100)
)
```

**흔한 실수**:
- ❌ fillna 누락으로 NaN 발생
- ❌ 오름차순으로 정렬

**실무 팁**:
긍정률 외에도 NPS(순추천지수)처럼 (긍정 - 부정) / 전체 지표도 많이 사용합니다.

---

### Q8. 누적 막대 차트로 감성 분포 시각화 ⭐⭐⭐⭐

**문제**: Q7의 데이터를 사용하여 제품별 감성 분포 누적 막대 차트를 그리세요.

In [ ]:
# Q8 정답

# Q7의 데이터 사용
product_reviews = pd.DataFrame({
    'product': ['A', 'A', 'A', 'B', 'B', 'B', 'C', 'C', 'C', 'C'],
    'sentiment': ['긍정', '긍정', '부정', '긍정', '부정', '부정', '긍정', '긍정', '긍정', '중립']
})

# 1. 제품별 감성 분포 집계
sentiment_dist = product_reviews.groupby(['product', 'sentiment']).size().unstack(fill_value=0)

# 2. 비율로 변환
sentiment_pct = sentiment_dist.div(sentiment_dist.sum(axis=1), axis=0) * 100

print("제품별 감성 비율(%)")
print(sentiment_pct.round(1))

# 3. 색상 및 순서 정의
colors = {'긍정': '#2ecc71', '부정': '#e74c3c', '중립': '#95a5a6'}
sentiment_order = ['긍정', '중립', '부정']

# 4. 누적 막대 차트 생성
fig = go.Figure()

for sentiment in sentiment_order:
    if sentiment in sentiment_pct.columns:
        fig.add_trace(go.Bar(
            x=sentiment_pct.index,
            y=sentiment_pct[sentiment],
            name=sentiment,
            marker_color=colors[sentiment],
            text=[f'{v:.0f}%' for v in sentiment_pct[sentiment]],
            textposition='inside'
        ))

# 5. 레이아웃 설정
fig.update_layout(
    title='제품별 감성 분포 (비율)',
    xaxis_title='제품',
    yaxis_title='비율 (%)',
    barmode='stack',
    legend_title='감성',
    yaxis=dict(range=[0, 100]),
    width=700,
    height=450
)

fig.show()

In [ ]:
# 테스트
assert fig is not None, "차트 생성 실패"
assert len(fig.data) >= 2, "감성 카테고리 부족"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. 피벗 테이블로 제품-감성 교차표 생성
2. 행별 비율 계산 (div 사용)
3. 감성별로 막대 추가 (barmode='stack')

**핵심 개념**:
- `unstack()`: 인덱스를 컬럼으로 변환
- `div(axis=0)`: 행별 나누기
- `barmode='stack'`: 누적 막대

**대안 솔루션**:
```python
# px.histogram 사용
fig = px.histogram(
    product_reviews, x='product', color='sentiment',
    barnorm='percent', barmode='stack'
)
```

**흔한 실수**:
- ❌ 개수 대신 비율로 변환 누락
- ❌ 감성 순서 미지정 (긍정 → 부정 순서가 직관적)

**실무 팁**:
누적 막대 차트에서는 긍정을 아래에, 부정을 위에 배치하면 비교가 쉽습니다.

---

### Q9. 시계열 감성 트렌드 분석 ⭐⭐⭐⭐

**문제**: 다음 일별 데이터를 사용하여 감성 점수 추이 라인 차트를 그리세요. 3일 이동평균도 함께 표시하세요.

In [ ]:
# Q9 정답

daily_data = pd.DataFrame({
    'date': pd.date_range('2024-01-01', periods=10),
    'sentiment_score': [0.3, 0.5, 0.2, -0.1, -0.3, 0.1, 0.4, 0.6, 0.3, 0.5]
})

# 1. 3일 이동평균 계산
daily_data['MA3'] = daily_data['sentiment_score'].rolling(window=3, min_periods=1).mean()

print("일별 감성 점수 및 이동평균")
print(daily_data)

# 2. 라인 차트 생성
fig = go.Figure()

# 일별 점수 (파란색 선 + 마커)
fig.add_trace(go.Scatter(
    x=daily_data['date'],
    y=daily_data['sentiment_score'],
    mode='lines+markers',
    name='일별 점수',
    line=dict(color='#3498db', width=2),
    marker=dict(size=8)
))

# 3일 이동평균 (빨간색 굵은 선)
fig.add_trace(go.Scatter(
    x=daily_data['date'],
    y=daily_data['MA3'],
    mode='lines',
    name='3일 이동평균',
    line=dict(color='#e74c3c', width=3)
))

# 3. y=0 기준선
fig.add_hline(y=0, line_dash='dash', line_color='gray')

# 4. 레이아웃 설정
fig.update_layout(
    title='일별 감성 점수 추이 (3일 이동평균)',
    xaxis_title='날짜',
    yaxis_title='감성 점수',
    yaxis=dict(range=[-1, 1]),
    width=800,
    height=450,
    hovermode='x unified'
)

fig.show()

In [ ]:
# 테스트
assert 'MA3' in daily_data.columns, "이동평균 컬럼 누락"
assert len(fig.data) == 2, "두 개의 트레이스가 있어야 함"
print("테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
1. rolling()으로 이동평균 계산
2. 원본 데이터와 이동평균을 각각 Scatter로 추가
3. 기준선으로 긍/부정 경계 표시

**핵심 개념**:
- `rolling(window=3)`: 3일 이동 윈도우
- `min_periods=1`: 최소 1개 데이터로도 계산 (초반 NaN 방지)
- `hovermode='x unified'`: 같은 x축 데이터 함께 표시

**대안 솔루션**:
```python
# px.line으로 기본 차트 후 add_trace
fig = px.line(daily_data, x='date', y='sentiment_score')
fig.add_scatter(x=daily_data['date'], y=daily_data['MA3'])
```

**흔한 실수**:
- ❌ min_periods 미지정으로 초반 NaN
- ❌ 두 선의 스타일 구분 없음

**실무 팁**:
이동평균 기간은 데이터 특성에 따라 3일, 7일, 30일 등 조정합니다. 주간 패턴이 있으면 7일을 권장합니다.

---

### Q10. 종합 VOC 분석 대시보드 ⭐⭐⭐⭐⭐

**문제**: VOC 데이터를 분석하고 종합 대시보드를 만드세요.

In [ ]:
# Q10 정답

np.random.seed(42)
voc_data = pd.DataFrame({
    'date': pd.date_range('2024-01-01', periods=20).repeat(3),
    'category': ['제품', '서비스', '배송'] * 20,
    'review': ['좋아요 만족합니다', '별로예요', '빨라요'] * 20,
    'sentiment_score': np.random.uniform(-1, 1, 60)
})

# 감성 분류 추가
voc_data['sentiment'] = voc_data['sentiment_score'].apply(
    lambda x: '긍정' if x > 0.1 else ('부정' if x < -0.1 else '중립')
)

print("VOC 데이터 미리보기")
print(voc_data.head(10))
print(f"\n총 리뷰 수: {len(voc_data)}개")

In [ ]:
# 1. 데이터 분석

# 전체 감성 분포
sentiment_counts = voc_data['sentiment'].value_counts()

# 카테고리별 평균 점수
category_scores = voc_data.groupby('category')['sentiment_score'].mean().sort_values()

# 일별 평균 점수
daily_scores = voc_data.groupby('date')['sentiment_score'].mean()

# 카테고리별 부정 비율
negative_rate = voc_data.groupby('category').apply(
    lambda x: (x['sentiment'] == '부정').sum() / len(x) * 100
).sort_values(ascending=False)

print("분석 결과 요약")
print("=" * 40)
print(f"\n감성 분포:\n{sentiment_counts}")
print(f"\n카테고리별 평균 점수:\n{category_scores}")
print(f"\n카테고리별 부정률(%):\n{negative_rate}")

In [ ]:
# 2. 종합 대시보드 생성

# 색상 정의
colors = {'긍정': '#2ecc71', '부정': '#e74c3c', '중립': '#95a5a6'}

# 2x2 서브플롯
fig = make_subplots(
    rows=2, cols=2,
    specs=[[{'type': 'pie'}, {'type': 'bar'}],
           [{'type': 'scatter'}, {'type': 'bar'}]],
    subplot_titles=('전체 감성 분포', '카테고리별 평균 점수',
                   '일별 감성 추이', '카테고리별 부정률')
)

# (1, 1) 파이 차트: 전체 감성 분포
fig.add_trace(
    go.Pie(
        labels=sentiment_counts.index,
        values=sentiment_counts.values,
        marker_colors=[colors[s] for s in sentiment_counts.index],
        hole=0.3,
        textinfo='percent+label'
    ),
    row=1, col=1
)

# (1, 2) 막대 차트: 카테고리별 평균 점수
bar_colors = ['#2ecc71' if x >= 0 else '#e74c3c' for x in category_scores.values]
fig.add_trace(
    go.Bar(
        x=category_scores.index,
        y=category_scores.values,
        marker_color=bar_colors,
        text=[f'{v:.2f}' for v in category_scores.values],
        textposition='outside',
        showlegend=False
    ),
    row=1, col=2
)

# (2, 1) 라인 차트: 일별 감성 추이
fig.add_trace(
    go.Scatter(
        x=daily_scores.index,
        y=daily_scores.values,
        mode='lines+markers',
        line=dict(color='#3498db', width=2),
        marker=dict(size=6),
        showlegend=False
    ),
    row=2, col=1
)

# (2, 2) 막대 차트: 카테고리별 부정률
fig.add_trace(
    go.Bar(
        x=negative_rate.index,
        y=negative_rate.values,
        marker_color='#e74c3c',
        text=[f'{v:.1f}%' for v in negative_rate.values],
        textposition='outside',
        showlegend=False
    ),
    row=2, col=2
)

# 레이아웃 설정
fig.update_layout(
    title_text='VOC 종합 분석 대시보드',
    height=800,
    width=1000,
    showlegend=True
)

# 축 레이블
fig.update_xaxes(title_text='카테고리', row=1, col=2)
fig.update_yaxes(title_text='감성 점수', row=1, col=2)
fig.update_xaxes(title_text='날짜', row=2, col=1)
fig.update_yaxes(title_text='감성 점수', row=2, col=1)
fig.update_xaxes(title_text='카테고리', row=2, col=2)
fig.update_yaxes(title_text='부정률 (%)', row=2, col=2)

# 기준선 추가
fig.add_hline(y=0, line_dash='dash', line_color='gray', row=1, col=2)
fig.add_hline(y=0, line_dash='dash', line_color='gray', row=2, col=1)

fig.show()

In [ ]:
# 3. 인사이트 도출

print("\n" + "=" * 60)
print("VOC 분석 인사이트")
print("=" * 60)

# 인사이트 1: 전체 감성 비율
positive_pct = sentiment_counts.get('긍정', 0) / len(voc_data) * 100
negative_pct = sentiment_counts.get('부정', 0) / len(voc_data) * 100
print(f"\n1. 전체 감성: 긍정 {positive_pct:.1f}% / 부정 {negative_pct:.1f}%")
if positive_pct > negative_pct:
    print("   → 전반적으로 긍정적인 반응")
else:
    print("   → 부정적 의견 비율이 높아 개선 필요")

# 인사이트 2: 가장 문제가 되는 카테고리
worst_category = negative_rate.idxmax()
worst_rate = negative_rate.max()
print(f"\n2. 개선 필요 카테고리: {worst_category} (부정률 {worst_rate:.1f}%)")
print(f"   → {worst_category} 관련 불만 원인 분석 필요")

# 인사이트 3: 시계열 트렌드
recent_avg = daily_scores[-5:].mean()
early_avg = daily_scores[:5].mean()
trend = "상승" if recent_avg > early_avg else "하락"
print(f"\n3. 감성 트렌드: {trend} 추세 (초반 {early_avg:.2f} → 최근 {recent_avg:.2f})")
if trend == "하락":
    print("   → 최근 부정 의견 증가, 원인 파악 시급")
else:
    print("   → 고객 만족도 개선 중")

In [ ]:
# 테스트
assert fig is not None, "대시보드 생성 실패"
assert len(fig.data) >= 4, "4개 이상의 서브플롯 필요"
print("테스트 통과! 종합 대시보드가 성공적으로 생성되었습니다.")

### 💡 풀이 설명

**접근 방법**:
1. 데이터 분석: 전체 분포, 카테고리별, 시계열, 부정률
2. make_subplots로 2x2 대시보드 구성
3. 각 차트에 맞는 시각화 타입 선택
4. 데이터 기반 인사이트 도출

**핵심 개념**:
- `make_subplots()`: 다중 차트 레이아웃
- `specs`: 각 셀의 차트 타입 지정
- 데이터 집계와 시각화 연결
- 정량적 인사이트 도출

**대안 솔루션**:
Plotly Dash를 사용하면 인터랙티브 필터링이 가능한 대시보드를 만들 수 있습니다.

**흔한 실수**:
- ❌ specs 타입과 trace 불일치
- ❌ 축 레이블, 제목 누락
- ❌ 인사이트 없이 차트만 제공

**실무 팁**:
대시보드는 시각화만이 아니라 "그래서 어떻게 해야 하는가?"라는 액션 아이템이 중요합니다. 항상 인사이트와 권장사항을 함께 제공하세요.

---

## 📊 학습 정리

### Part 1: 사전 기반 감성 분석 핵심 요약

| 개념 | 도구/방법 | 활용 |
|------|----------|------|
| VADER | `SentimentIntensityAnalyzer` | 영어 SNS 텍스트 분석 |
| 한국어 사전 | KNU 감성 사전, 커스텀 사전 | 한글 리뷰 분석 |
| 감성 분류 | compound 점수 기준 | 긍정/부정/중립 |
| 강조어 처리 | 점수 배수 적용 | "매우 좋다" 처리 |
| 부정어 처리 | 점수 반전 | "안 좋다" 처리 |

### Part 2: 감성 분석 시각화 핵심 요약

| 시각화 | Plotly 함수 | 언제 쓰나? |
|--------|------------|----------|
| 파이 차트 | `px.pie()` | 감성 분포 비율 |
| 막대 차트 | `go.Bar()` | 카테고리별 비교 |
| 누적 막대 | `barmode='stack'` | 구성 비율 비교 |
| 라인 차트 | `go.Scatter()` | 시계열 트렌드 |
| 대시보드 | `make_subplots()` | 종합 분석 |

### 💡 실무 팁

1. **도메인 특화 사전**: 일반 사전 + 도메인 단어로 정확도 향상
2. **강조어/부정어**: 반드시 처리해야 정확한 분석 가능
3. **평점 vs 감성**: 상관관계 확인하여 분석 신뢰도 검증
4. **시계열 분석**: 이동평균으로 노이즈 제거, 트렌드 파악
5. **VOC 대시보드**: 한눈에 현황 파악, 개선점 도출
6. **ML 병행**: 사전 기반으로 초기 분석, ML로 정확도 개선
7. **실시간 모니터링**: 부정 급증 시 알림 설정 권장